In [51]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("Movies Analysis").getOrCreate()

# Φόρτωση του dataset
df = spark.read.json("movie.json")

# Εμφάνιση του σχήματος του DataFrame για να κατανοήσουμε τα δεδομένα
df.printSchema()


root
 |-- _corrupt_record: string (nullable = true)
 |-- actors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- countries: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- directors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genre: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- imdb_url: string (nullable = true)
 |-- img_url: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- metascore: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- users_rating: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- year: string (nullable = true)



In [68]:
df.dtypes

[('_corrupt_record', 'string'),
 ('actors', 'array<string>'),
 ('countries', 'array<string>'),
 ('description', 'string'),
 ('directors', 'array<string>'),
 ('genre', 'array<string>'),
 ('imdb_url', 'string'),
 ('img_url', 'string'),
 ('languages', 'array<string>'),
 ('metascore', 'int'),
 ('rating', 'string'),
 ('runtime', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('users_rating', 'float'),
 ('votes', 'int'),
 ('year', 'int')]

In [54]:
from pyspark.sql.types import IntegerType, FloatType

# Convert the 'metascore' column to IntegerType
df = df.withColumn("metascore", df["metascore"].cast(IntegerType()))

# Convert the 'users_rating' column to FloatType
df = df.withColumn("users_rating", df["users_rating"].cast(FloatType()))

# Convert the 'votes' column to IntegerType
df = df.withColumn("votes", df["votes"].cast(IntegerType()))

# Convert the 'year' column to IntegerType
df = df.withColumn("year", df["year"].cast(IntegerType()))

# Re-display data types to confirm the changes
df.dtypes



[('_corrupt_record', 'string'),
 ('actors', 'array<string>'),
 ('countries', 'array<string>'),
 ('description', 'string'),
 ('directors', 'array<string>'),
 ('genre', 'array<string>'),
 ('imdb_url', 'string'),
 ('img_url', 'string'),
 ('languages', 'array<string>'),
 ('metascore', 'int'),
 ('rating', 'string'),
 ('runtime', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('users_rating', 'float'),
 ('votes', 'int'),
 ('year', 'int')]

Data Schema and Type Conversion:

Initially, we displayed the schema of the dataset to understand the structure and types of data available.
This is a crucial step for data exploration and ensuring that the dataset contains the expected fields and data types.
We've successfully converted the data types of several columns (metascore, users_rating, votes, and year) to more appropriate types for analysis, such as integers and floats. 
This type conversion is important for enabling numerical operations on these fields.



In [55]:
df.select(["title", "votes", "users_rating"]).where(df.votes.isNull() | df.users_rating.isNull()).show()


+--------------------+-----+------------+
|               title|votes|users_rating|
+--------------------+-----+------------+
|                null| null|        null|
|A Rainy Day in Ne...| null|         6.6|
|Ferris Bueller's ...| null|         7.8|
|           Cape Fear| null|         7.3|
|          Mean Girls| null|         7.0|
|     Gretel & Hansel| null|         5.3|
|Jurassic World: F...| null|         6.2|
|        The Avengers| null|         8.0|
|   Good Will Hunting| null|         8.3|
|2001: A Space Ody...| null|         8.3|
|            Scarface| null|         8.3|
|  Dazed and Confused| null|         7.6|
|           Our House| null|         5.3|
|   Full Metal Jacket| null|         8.3|
|Harry Potter and ...| null|         7.4|
|         Gran Torino| null|         8.1|
|Three Billboards ...| null|         8.2|
|           Cast Away| null|         7.8|
|Star Wars: Episod...| null|         7.5|
|            Bird Box| null|         6.6|
+--------------------+-----+------

Handling Missing Values:
By selecting movies where either votes or users_rating is null and showing them, we've identified missing values in wer dataset. 
Then, we've used the .na.fill() method to fill missing votes with 0, which is a common practice to handle null values for numerical analysis.
Filtering and Analysis:



In [56]:
df_fil = df.na.fill({"votes": 0})


In [57]:
df_fil.select(["title", "votes", "users_rating"]).where(df.votes.isNull() | df.users_rating.isNull()).show()


+--------------------+-----+------------+
|               title|votes|users_rating|
+--------------------+-----+------------+
|                null|    0|        null|
|A Rainy Day in Ne...|    0|         6.6|
|Ferris Bueller's ...|    0|         7.8|
|           Cape Fear|    0|         7.3|
|          Mean Girls|    0|         7.0|
|     Gretel & Hansel|    0|         5.3|
|Jurassic World: F...|    0|         6.2|
|        The Avengers|    0|         8.0|
|   Good Will Hunting|    0|         8.3|
|2001: A Space Ody...|    0|         8.3|
|            Scarface|    0|         8.3|
|  Dazed and Confused|    0|         7.6|
|           Our House|    0|         5.3|
|   Full Metal Jacket|    0|         8.3|
|Harry Potter and ...|    0|         7.4|
|         Gran Torino|    0|         8.1|
|Three Billboards ...|    0|         8.2|
|           Cast Away|    0|         7.8|
|Star Wars: Episod...|    0|         7.5|
|            Bird Box|    0|         6.6|
+--------------------+-----+------

this code effectively filters movies with a specific number of votes and displays titles and vote counts, showcasing Spark's capability to handle conditional data filtering and presentation.
We've also identified the movie with the lowest user rating that starts with a specific letter and has a significant number of votes, providing insight into specific segments of wer dataset.
Additionally, calculating the average users_rating and metascore for movies starting with a specific letter offers valuable insights into the dataset's characteristics,
highlighting the use of aggregation functions in Spark.



In [58]:
df_fil.filter(df_fil.votes >= 200).select("title", "votes").show()


+--------------------+-----+
|               title|votes|
+--------------------+-----+
|     Debt Collectors|  562|
|       Rogue Warfare|  764|
|  The Animal Kingdom|  940|
|         Palm Swings|  640|
|        Purity Falls|  308|
|    Assassin 33 A.D.|  920|
|             Ecstasy|  281|
|       Stallion Road|  205|
|      Hollywood Dirt|  925|
| Always a Bridesmaid|  386|
|          Dead Water|  475|
| Bullet for a Badman|  635|
|           Back Fork|  279|
|            Mob Town|  330|
|The Dallas Connec...|  674|
|Primal: Tales of ...|  465|
|         Chatterbox!|  559|
|          Georgetown|  392|
|    A Champion Heart|  238|
|        The Sex Trip|  365|
+--------------------+-----+
only showing top 20 rows



In [59]:
#It returns the “title”, “year”, and “users_rating” of the movie with the lowest
#“users_rating” that its title starts with the first letter of your first name and it has at least 200,000 votes. 

lowest_rated_movie = df_fil.filter(df_fil.title.startswith("E") & (df_fil.votes >= 200))\
    .orderBy("users_rating", ascending=True)\
    .select("title", "year", "users_rating")\
    .limit(1)

lowest_rated_movie.show()

+---------------+----+------------+
|          title|year|users_rating|
+---------------+----+------------+
|Evil Behind You|2006|         1.8|
+---------------+----+------------+



In [60]:
# It returns the average “users_rating” and “metascore” of the movies that their title  starts with the *second* letter of your first name and for which there are at least  200,000 votes.
#  the second letter of your first name is 'L'

from pyspark.sql.functions import avg

average_ratings = df_fil.filter(df_fil.title.startswith("L") & (df_fil.votes >= 200)) \
    .agg(avg("users_rating").alias("average_users_rating"), 
         avg("metascore").alias("average_metascore"))

average_ratings.show()


+--------------------+-----------------+
|average_users_rating|average_metascore|
+--------------------+-----------------+
|   5.403218887650404|            48.02|
+--------------------+-----------------+



In [62]:
#It returns the “title”, “year”, and “votes” of the movie with the most votes, when only movies with title starting with the *third* letter of your first name are considered. 
#  the third letter of your first name is 'E'

most_votes_movie = df_fil.filter(df_fil.title.startswith("E")) \
    .orderBy("votes", ascending=False) \
    .select("title", "year", "votes") \
    .limit(1)

most_votes_movie.show()


+----------------+----+-----+
|           title|year|votes|
+----------------+----+-----+
|Executive Target|1997|  994|
+----------------+----+-----+



Advanced Filtering and Aggregation:
Finding the movie with the most votes that starts with a certain letter demonstrates advanced filtering capabilities and sorting based on specific conditions.
We analysis of the maximum and minimum users_rating for movies involving a specific actor (Bruce Willis in this case) 
This could provide insights into the actor's career highs and lows according to audience ratings.

In [64]:
#It returns the maxium and minimum users_rating score for all movies into which Bruce Willis is involved as an actor
from pyspark.sql.functions import array_contains, max, min

# Αναζήτηση ταινιών με τον Bruce Willis και υπολογισμός της μέγιστης και ελάχιστης βαθμολογίας χρηστών
bruce_willis_movies = df_fil.filter(array_contains(df_fil.actors, "Bruce Willis")) \
    .agg(max("users_rating").alias("max_users_rating"), 
         min("users_rating").alias("min_users_rating"))

bruce_willis_movies.show()


+----------------+----------------+
|max_users_rating|min_users_rating|
+----------------+----------------+
|             8.9|             3.6|
+----------------+----------------+

